The aim of this notebook is to make use of the word2vec model to find similar songs

In [1]:
import pandas as pd
import numpy as np
import gensim.models.word2vec as w2v
import multiprocessing
import os
import re
import pprint
import sklearn.manifold
import matplotlib.pyplot as plt

unable to import 'smart_open.gcs', disabling that module


In [2]:
!pip install smart_open.gcs

  ERROR: Could not find a version that satisfies the requirement smart_open.gcs (from versions: none)
ERROR: No matching distribution found for smart_open.gcs


Though non english artists were removed, the dataset contained Hindi lyrics of Lata Mangeshkar written in English. Therefore, I decided to remove all songs sung by her.

In [3]:
songs = pd.read_csv("hhgroups_merge_28_05.csv", header=0, encoding='utf-8')
#songs.head()
songs = songs[songs.artista != 'Lata Mangeshkar']
songs.head()

,id,artista,cancion,album,letra,anyo,visitas
0,0,Denom,Machete (con Jarfaiter y Gente jodida),Medicina,"Para su nuevo disco ""Medicina"", Denom ha vuelt...",2019,126
1,1,Denom,Vacío (con Ivo Incuerdo),Medicina,"[Denom]\nYo que quería, yo que pedía vida,\nSe...",2019,361
2,2,Denom,El orgullo es fiel (con Juancho Marqués y Elio...,Medicina,"""El orgullo es fiel"" es uno de los cortes incl...",2019,262
3,3,Denom,Mueve mueve (con Fernandocosta),Medicina,"[Estribillo: Denom] (x2)\nMueve, mueve, mueve,...",2019,578
4,4,Jaro Desperdizio,Insomnia,"Sin álbum, es un vídeo suelto","[Estribillo]\nY en esta noche, ¿Quién me arrop...",2019,219


To train the word2vec model, we first need to build its vocabulary. To do that, I iterated over each song and added it to an array that can later be fed to the model.

In [4]:
text_corpus = []
for song in songs['letra']:
    words = song.lower().split()
    text_corpus.append(words)


# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7


downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

songs2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

songs2vec.build_vocab(text_corpus)
print (len(text_corpus))

9325


In [5]:
import time
start_time = time.time()



songs2vec.train(text_corpus, total_examples=songs2vec.corpus_count, epochs=2)

if not os.path.exists("trained"):
    os.makedirs("trained")

songs2vec.save(os.path.join("trained", "songs2vectors.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 67.5272262096405 seconds ---


In [6]:
songs2vec = w2v.Word2Vec.load(os.path.join("trained", "songs2vectors.w2v"))

#### Let's explore our model

Find similar words

In [7]:
songs2vec.wv.most_similar("viajar")

[('volverme', 0.9679238796234131),
 ('andar,', 0.9619711637496948),
 ('marcharme', 0.9589545726776123),
 ('echarme', 0.9576978087425232),
 ('gritar', 0.9571213722229004),
 ('andar', 0.9570766091346741),
 ('descansar', 0.9567746520042419),
 ('recorrer', 0.955890417098999),
 ('encontrarme', 0.9552344083786011),
 ('condenado', 0.954977810382843)]

In [8]:
songs2vec.wv.most_similar("vacaciones")

[('rama,', 0.9674484729766846),
 ('pegando', 0.9669114947319031),
 ('cargando', 0.9653007388114929),
 ('círculos', 0.9640078544616699),
 ('catorce', 0.9639139175415039),
 ('playas', 0.9633513689041138),
 ('bajé', 0.9630541205406189),
 ('nadando', 0.9627984166145325),
 ('trance,', 0.9622144103050232),
 ('chocar', 0.9618642926216125)]

Words out of context

In [9]:
songs2vec.wv.doesnt_match("te quiero".split())

C:\Users\merxe\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'te'

In [10]:
songs2vec.most_similar(positive=['mujer', 'rey'], negative=['hombre'])


C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('hija', 0.8728294372558594),
 ('gorra', 0.8667598962783813),
 ('cerda', 0.8623393177986145),
 ('fallaste,', 0.8594537973403931),
 ('fiera,', 0.8585516214370728),
 ('fan', 0.8553972244262695),
 ('peli', 0.8533725142478943),
 ('fiera', 0.8533666133880615),
 ('reina', 0.853366494178772),
 ('nariz', 0.8510406017303467)]

Semantic distance between words

In [11]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = songs2vec.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [12]:
nearest_similarity_cosmul("españa", "mar", "tristeza")

españa es a mar, lo que cultura es a tristeza


With the word vector embeddings in place, it is now time to calculate the normalised vector sum of each song. This process can take some time since it has to be done for each of 57,000 songs.

In [13]:
print(songs2vec['noche'])
def songVector(row):
    vector_sum = 0
    words = row.lower().split()
    for word in words:
        vector_sum = vector_sum + songs2vec[word]
    vector_sum = vector_sum.reshape(1,-1)
    normalised_vector_sum = sklearn.preprocessing.normalize(vector_sum)
    return normalised_vector_sum


import time
start_time = time.time()

songs['song_vector'] = songs['letra'].apply(songVector)




C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


[ 0.7446445  -0.23183636  0.43731368  0.5191476   0.49625272 -0.18808517
 -0.68191165 -0.8255139   0.64113164 -0.04547158 -0.11215883  0.74431765
 -0.06482858 -0.17128032  0.07728334 -0.28827518 -0.3667322  -1.0101421
  0.33123535 -0.5934202  -0.24798942 -0.9812952   0.12943825 -0.9393366
 -0.33045885 -0.1939345  -0.06186122 -0.2759384  -1.2173567   0.02517581
 -0.01867697 -0.20862836 -0.27560672  0.29629606  0.22532918 -0.18702564
  0.6653837  -0.8046098   0.09064052  0.09128334 -0.05555522 -0.10120914
 -0.3822116   0.4650935  -0.01988475 -0.65692246  0.20587641  0.03594383
  0.47884375  0.8037719 ]


C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

...

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

**t-sne and random song selection** 

The songs have 50 dimensions each. Application of t-sne is memory intensive and hence it is slightly easier on the computer to use a random sample of the 57,000 songs.

In [14]:
song_vectors = []
from sklearn.model_selection import train_test_split

train, test = train_test_split(songs, test_size = 0.9)


for song_vector in train['song_vector']:
    song_vectors.append(song_vector)

train.head(10)

,id,artista,cancion,album,letra,anyo,visitas,song_vector
1565,25,Rapsusklei,Te quiero,Pandemia,"Y es que te quiero.\nLo tuyo es diferente, y c...",2010,142186,"[[0.12662572, 0.07173827, 0.100629255, 0.08490..."
4380,35,Chuty,Bitcoin,Bitcoin,"[Estribillo]\nBitcoin, bitcoin,\nNo apostaron ...",2019,4887,"[[0.10730026, 0.07199616, 0.08055281, 0.069071..."
3003,33,Porta,Mi realidad,No es cuestion de edades,No podemos demostrar que lo que vivimos es rea...,2006,4182,"[[0.13214205, 0.07942772, 0.084953114, 0.08078..."
9163,3,Rapsoda,Apocalipsis,La sombra de la verdad,"Bienvenidos al fin de nuestra era,\nse muere l...",2012,818,"[[0.13510205, 0.07232712, 0.06188844, 0.096423..."
8412,22,ZPU,Primera clase (con Soprano),He tenido un sueño,"[ZPU]\nWelcome to mi primera clase,\nYa sé lo ...",2010,1669,"[[0.10117202, 0.058134556, 0.06213877, 0.06046..."
1575,35,Lèolo,Poema,El proyecto venus,Preguntas si puedes volar a quienes cortan tus...,2010,3887,"[[0.10936254, 0.06840263, 0.10268898, 0.085170..."
4423,23,XPi,Que dirían,Sin álbum,[Estribillo]\nQue dirían\nsi les dijera que ví...,0,1831,"[[0.124226816, 0.084140964, 0.07149013, 0.0990..."
1034,44,Mowlihawk,Hice cosas mal (con Porta y Patri),Torre de control,[Mowlihawk]\nAhora estoy más centrado pero me ...,2011,14968,"[[0.12148382, 0.07581923, 0.100164995, 0.08026..."
7768,38,Geiser,Live weather,Introspección,"Me eché una carrera contra el tiempo,\nPara po...",2015,463,"[[0.12392651, 0.069072366, 0.103916615, 0.0932..."
3685,0,Yamal,Acto inicial,Sin álbum,"[Yamal]\nLlega Yamal,las rimas finas te las ti...",0,958,"[[0.12869972, 0.054052234, 0.10438215, 0.08884..."


I had a fairly measly 4gb machine and wasn't able to generate a more accurate model. However, one can play around with the number of iterations, learning rate and other factors to fit the model better. If you have too many dimensions (~300+), it might make sense to use PCA first and then t-sne.

In [15]:
X = np.array(song_vectors).reshape((932, 50))

start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, n_iter=250, random_state=0, verbose=2)

all_word_vectors_matrix_2d = tsne.fit_transform(X)

print("--- %s seconds ---" % (time.time() - start_time))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 932 samples in 0.003s...
[t-SNE] Computed neighbors for 932 samples in 0.111s...
[t-SNE] Computed conditional probabilities for sample 932 / 932
[t-SNE] Mean sigma: 0.028795
[t-SNE] Computed conditional probabilities in 0.098s
[t-SNE] Iteration 50: error = 74.2213516, gradient norm = 0.3424574 (50 iterations in 0.663s)
[t-SNE] Iteration 100: error = 74.3623276, gradient norm = 0.3392808 (50 iterations in 0.597s)
[t-SNE] Iteration 150: error = 76.4866867, gradient norm = 0.3090650 (50 iterations in 0.520s)
[t-SNE] Iteration 200: error = 76.4409409, gradient norm = 0.3003353 (50 iterations in 0.606s)
[t-SNE] Iteration 250: error = 76.8755188, gradient norm = 0.3082820 (50 iterations in 0.748s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 76.875519
[t-SNE] KL divergence after 251 iterations: 17976931348623157081452742373170435679807056752584499659891747680315726078002853876058955863276687817154045895351438246

In [16]:
df=pd.DataFrame(all_word_vectors_matrix_2d,columns=['X','Y'])

df.head(10)

train.head()

df.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

Joining two dataframes to obtain each song's corresponding X,Y co-ordinate.

In [17]:
two_dimensional_songs = pd.concat([train, df], axis=1)

two_dimensional_songs.head()

,id,artista,cancion,album,letra,anyo,visitas,song_vector,X,Y
0,25,Rapsusklei,Te quiero,Pandemia,"Y es que te quiero.\nLo tuyo es diferente, y c...",2010,142186,"[[0.12662572, 0.07173827, 0.100629255, 0.08490...",-0.164684,-0.221583
1,35,Chuty,Bitcoin,Bitcoin,"[Estribillo]\nBitcoin, bitcoin,\nNo apostaron ...",2019,4887,"[[0.10730026, 0.07199616, 0.08055281, 0.069071...",-0.149111,-0.060332
2,33,Porta,Mi realidad,No es cuestion de edades,No podemos demostrar que lo que vivimos es rea...,2006,4182,"[[0.13214205, 0.07942772, 0.084953114, 0.08078...",-0.219862,-0.080158
3,3,Rapsoda,Apocalipsis,La sombra de la verdad,"Bienvenidos al fin de nuestra era,\nse muere l...",2012,818,"[[0.13510205, 0.07232712, 0.06188844, 0.096423...",0.265941,0.186624
4,22,ZPU,Primera clase (con Soprano),He tenido un sueño,"[ZPU]\nWelcome to mi primera clase,\nYa sé lo ...",2010,1669,"[[0.10117202, 0.058134556, 0.06213877, 0.06046...",0.530611,0.339056


**Plotting the results**

Using plotly, I plotted the results so that it becomes easier to explore similar songs based on their colors and clusters.

In [18]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

trace1 = go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista'],
    mode='markers',
    marker=dict(
        size= 5,#'7',
        color = np.random.randn(5717), #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

iplot(data)

In [19]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista']+ "_"+two_dimensional_songs['cancion'] ,
    mode='markers',
    marker=dict(
        size= 10,#'7',
        color = np.random.randn(5717), #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
))

fig.show()

In [20]:
## LOOK FOR COMMON SONGS AND ANALYZE THE TEXT

In [21]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(
    y = np.random.randn(500),
    mode='markers',
    marker=dict(
        size=16,
        color=np.random.randn(500), #set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        showscale=True
    )
))

fig.show()

In [22]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
              color='species')
fig.show()